We will focuses on 5 core IBCS rules for charts clarity.
- Scaling – Avoid misleading charts by keeping scales consistent and starting at zero when required.

- Titles – Every chart should be self-explanatory without needing extra context.

- Color Usage – Use color intentionally to highlight important information, not as decoration.

- Axis Formatting – Make axes readable, properly labeled, and uncluttered.

- Clutter Reduction – Remove any visual elements that do not support understanding.



In [ ]:
import json

def generate_feedback(rule: str, label: str, confidence: float, details: dict = None) -> dict:
    feedback = {
        "label": label,
        "confidence": round(confidence, 2),
        "feedback": []
    }
    
    # If compliant, keep it short
    if label == 'compliant':
        feedback["feedback"].append(
            f"Great! It follows the {rule} rules. "
            f"Confidence: {confidence:.0%}."
        )
        return feedback
    
    # Non-compliant: Give specific, helpful suggestions
    suggestions = []
    
    if rule == 'scaling':
        suggestions.extend([
            "**Start bar charts at zero.** If your y-axis starts above zero, it can make small differences look huge.",
            "**Use the same scale for similar charts.** If you're comparing sales across regions, all charts should have identical y-axis ranges.",
            "**Show scale breaks clearly.** If you must skip part of the scale, use a visible break symbol (like ~) so readers know.",
            "**Label your units.** Add '(in EUR)', '(%)' or similar to your axis labels.",
            "**Avoid dual axes unless absolutely necessary.** Two different scales on one chart confuse readers."
        ])
        
        # Add specific issues from Grad-CAM
        if details:
            violations = details.get('violations', [])
            if 'axis_misaligned' in violations:
                suggestions.insert(0, "**Axis alignment issue detected.** Check the areas highlighted in red.")
            if 'non_zero_start' in violations:
                suggestions.insert(0, "**Your y-axis doesn't start at zero.** This can mislead viewers.")
            if 'inconsistent_scale' in violations:
                suggestions.insert(0, "**Different charts use different scales.** Make them uniform for fair comparison.")
    
    elif rule == 'titles':
        suggestions.extend([
            "**Use descriptive titles.** Good example: 'Monthly Revenue (EUR) - Q1 2025'. Bad example: 'Chart 1'.",
            "**Include the 5 W's:** What (Revenue), Where (Netherlands), When (January 2025), hoW much (in thousands).",
            "**Put titles at the top** of each chart, not inside it.",
            "**Keep it concise but clear.** Aim for one line if possible."
        ])
        
        if details and 'missing_title' in details.get('violations', []):
            suggestions.insert(0, "**Missing title detected.** Every chart needs a clear heading.")
    
    elif rule == 'color_misuse':
        suggestions.extend([
            "**Use color sparingly.** Only highlight what matters—usually negatives (red) or key data points.",
            "**Stick to IBCS colors:** Blue/grey for normal data, red for negative, green for positive variance.",
            "**Avoid rainbow charts.** Too many colors make it hard to focus.",
            "**Test in grayscale.** If your chart doesn't make sense in black and white, you're relying too much on color."
        ])
        
        if details and 'excessive_colors' in details.get('violations', []):
            suggestions.insert(0, "**Too many colors detected.** Simplify your color palette.")
    
    elif rule == 'axis_check':
        suggestions.extend([
            "**Label both axes clearly.** Include units like '(thousands)', '(%)' or '(days)'.",
            "**Use readable tick marks.** Not too many (cluttered) or too few (unclear).",
            "**Rotate labels if needed.** Long category names work better at 45° or vertically.",
            "**Remove unnecessary gridlines.** Keep only horizontal lines for bar charts, only vertical for column charts."
        ])
        
        if details and 'missing_units' in details.get('violations', []):
            suggestions.insert(0, "**Missing units on axis.** Add '(EUR)', '(%)' etc. to your labels.")
    
    elif rule == 'clutter_detection':
        suggestions.extend([
            "**Remove decorative elements.** 3D effects, shadows, and borders don't add value.",
            "**Delete redundant legends.** If you only have one data series, label it directly on the chart.",
            "**Simplify backgrounds.** Use white or light grey—no patterns or gradients.",
            "**Cut unnecessary labels.** If every bar is labeled, you don't need y-axis tick marks."
        ])
        
        if details and 'excessive_elements' in details.get('violations', []):
            suggestions.insert(0, "**Too many visual elements.** Simplify for better readability.")
    
    else:
        suggestions.append(f"Unknown rule: '{rule}'. Please check your configuration.")
    
    # Add summary at the end
    suggestions.append(
        f"\n**Confidence: {confidence:.0%}** | "
        f"Check the highlighted areas in the visualization for specific issues."
    )
    
    feedback["feedback"] = suggestions
    return feedback

In [5]:
# Test the function
if __name__ == "__main__":
    # Test 1: Non-compliant scaling with violations
    result1 = generate_feedback(
        rule='scaling',
        label='non-compliant',
        confidence=0.87,
        details={'violations': ['non_zero_start', 'inconsistent_scale']}
    )
    print("Test 1 - Scaling Issues:")
    print(json.dumps(result1, indent=2))
    
    print("\n" + "="*60 + "\n")
    
    # Test 2: Compliant
    result2 = generate_feedback(
        rule='scaling',
        label='compliant',
        confidence=0.92
    )
    print("Test 2 - Compliant:")
    print(json.dumps(result2, indent=2))
    
    print("\n" + "="*60 + "\n")
    
    # Test 3: Color misuse
    result3 = generate_feedback(
        rule='color_misuse',
        label='non-compliant',
        confidence=0.78,
        details={'violations': ['excessive_colors']}
    )
    print("Test 3 - Color Issues:")
    print(json.dumps(result3, indent=2))

Test 1 - Scaling Issues:
{
  "label": "non-compliant",
  "confidence": 0.87,
  "feedback": [
    "**Different charts use different scales.** Make them uniform for fair comparison.",
    "**Your y-axis doesn't start at zero.** This can mislead viewers.",
    "**Start bar charts at zero.** If your y-axis starts above zero, it can make small differences look huge.",
    "**Use the same scale for similar charts.** If you're comparing sales across regions, all charts should have identical y-axis ranges.",
    "**Show scale breaks clearly.** If you must skip part of the scale, use a visible break symbol (like ~) so readers know.",
    "**Label your units.** Add '(in EUR)', '(%)' or similar to your axis labels.",
    "**Avoid dual axes unless absolutely necessary.** Two different scales on one chart confuse readers.",
    "\n**Confidence: 87%** | Check the highlighted areas in the visualization for specific issues."
  ]
}


Test 2 - Compliant:
{
  "label": "compliant",
  "confidence": 0.92,
 